# Learn LangGraph: The Weather Map Example

This code is a walk through from [Learn LangGraph - The Easy Way](https://youtu.be/R8KB-Zcynxc?si=M5RDpfIdMA-cvisY) and the code found at [code](https://github.com/menloparklab/LangGraphJourney/blob/main/LangGraphLearning.ipynb)

This tutorial offers a step-by-step guide to building agent applications using LangGraph, a library offered by LangChain. Starting with an overview of LangGraph and its benefits, the tutorial first explores how to make a simple graph and create nodes. It goes on to explain how to create functions and run within nodes. It also covers importing LangChain tools, binding them to the model, how to parse information from nodes, and setting up a state graph. Finally, the tutorial covers adding a conditional edge to the graph and demonstrates running the graph using user inputs.

In [34]:
from dotenv import find_dotenv, load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.utilities import OpenWeatherMapAPIWrapper

load_dotenv(find_dotenv())

True

In [35]:
weather = OpenWeatherMapAPIWrapper()
model = ChatOpenAI(temperature=0) 


Nodes as function calls, that returns a value and sends it to the next stage.

In [36]:
def function_agent(input_1):
    complete_query = "Your task is to provide only the city name based on the user query. \
        Nothing more, just the city name mentioned. Following is the user query: " + input_1
    response = model.invoke(complete_query)
    return response.content

def function_2(input_2):
    return "Agent Says: " + input_2

In [37]:
from langgraph.graph import Graph

# Define a Langchain graph
workflow = Graph()

#calling node 1 as agent
workflow.add_node("agent", function_agent)
workflow.add_node("node_2", function_2)

workflow.add_edge('agent', 'node_2')

workflow.set_entry_point("agent")
workflow.set_finish_point("node_2")

app = workflow.compile()

In [38]:
weather_data = weather.run("Asbury Park, NJ, USA")
print(weather_data)

In Asbury Park, NJ, USA, the current weather is as follows:
Detailed status: few clouds
Wind speed: 4.12 m/s, direction: 180°
Humidity: 35%
Temperature: 
  - Current: 25.12°C
  - High: 26.57°C
  - Low: 23.53°C
  - Feels like: 24.6°C
Rain: {}
Heat index: None
Cloud cover: 20%


In [43]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]



In [44]:
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_community.tools.openweathermap import OpenWeatherMapQueryRun
from langchain_core.utils.function_calling import convert_to_openai_function

tools = [OpenWeatherMapQueryRun()]

model = ChatOpenAI(temperature=0, streaming=True)
functions = [convert_to_openai_function(t) for t in tools]
model = model.bind_functions(functions)

In [46]:
# the task is to parse out the city name from the user query
def function_1(state):
    messages = state['messages']
    response = model.invoke(messages)
    return {"messages": [response]}

from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage
from langgraph.prebuilt import ToolExecutor

tool_executor = ToolExecutor(tools)

def function_2(state):
    messages = state['messages']
    last_message = messages[-1] # this has the query we need to send to the tool provided by the agent

    parsed_tool_input = json.loads(last_message.additional_kwargs["function_call"]["arguments"])

    # We construct an ToolInvocation from the function_call and pass in the tool name and the expected str input for OpenWeatherMap tool
    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=parsed_tool_input['__arg1'],
    )
    
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)

    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(content=str(response), name=action.tool)

    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}

def function_3(state):
    messages = state['messages']
    user_input = messages[0]
    available_info = messages[-1]
    agent2_query = "Your task is to provide info concisely based on the user query and the available information from the internet. \
                        Following is the user query: " + user_input + " Available information: " + available_info
    response = model.invoke(agent2_query)
    return response.content

In [47]:
def where_to_go(state):
    messages = state['messages']
    last_message = messages[-1]
    
    if "function_call" in last_message.additional_kwargs:
        return "continue"
    else:
        return "end"

In [53]:
# import StateGraph and pass AgentState to it
from langgraph.graph import StateGraph, END
workflow = StateGraph(AgentState)

workflow.add_node("agent", function_1)
workflow.add_node("tool", function_2)

# The conditional edge requires the following info below.
# First, we define the start node. We use `agent`.
# This means these are the edges taken after the `agent` node is called.
# Next, we pass in the function that will determine which node is called next, in our case where_to_go().


# Based on the return from where_to_go, we define the next nodes to be called. If the return is "continue" then we call the tool node.
# Otherwise we finish. END is a special node marking that the graph should finish.
workflow.add_conditional_edges("agent", where_to_go,{"continue": "tool", "end": END})

# We now add a normal edge from `tools` to `agent`.
# This means that if `tool` is called, then it has to call the 'agent' next. 
workflow.add_edge('tool', 'agent')

# Basically, agent node has the option to call a tool node based on a condition, 
# whereas tool node must call the agent in all cases based on this setup.

workflow.set_entry_point("agent")

app = workflow.compile()


In [49]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="what is the temperature in las vegas")]}
app.invoke(inputs)

{'messages': [HumanMessage(content='what is the temperature in las vegas'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"__arg1":"Las Vegas"}', 'name': 'open_weather_map'}}, response_metadata={'finish_reason': 'function_call'}, id='run-96abedb8-53aa-4160-9d15-189474a8b1ca-0'),
  FunctionMessage(content='In Las Vegas, the current weather is as follows:\nDetailed status: clear sky\nWind speed: 4.92 m/s, direction: 60°\nHumidity: 20%\nTemperature: \n  - Current: 17.3°C\n  - High: 19.31°C\n  - Low: 15.93°C\n  - Feels like: 15.61°C\nRain: {}\nHeat index: None\nCloud cover: 0%', name='open_weather_map'),
  AIMessage(content='The current temperature in Las Vegas is 17.3°C. It is a clear sky with a humidity of 20%. The wind speed is 4.92 m/s.', response_metadata={'finish_reason': 'stop'}, id='run-013639b2-2f6e-48fa-9f40-a09b355ceb16-0')]}

In [51]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="How are you?")]}
app.invoke(inputs)

{'messages': [HumanMessage(content='How are you?'),
  AIMessage(content="I'm just a computer program, so I don't have feelings, but I'm here to help you with any questions or tasks you have. How can I assist you today?", response_metadata={'finish_reason': 'stop'}, id='run-45e951e1-35b1-4796-a166-1da8164ce718-0')]}

## Explanation Use Only

Run the cell if you want to review the contents in each of the steps.

In [56]:
from langchain_core.messages import HumanMessage
inputs = {"messages": [HumanMessage(content="what is the temperature in las vegas")]}
for output in app.stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'agent':
---
{'messages': [AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"__arg1":"Las Vegas"}', 'name': 'open_weather_map'}}, response_metadata={'finish_reason': 'function_call'}, id='run-db9d1535-a626-4c5c-9fce-9b14786329ea-0')]}

---

Output from node 'tool':
---
{'messages': [FunctionMessage(content='In Las Vegas, the current weather is as follows:\nDetailed status: clear sky\nWind speed: 4.92 m/s, direction: 60°\nHumidity: 19%\nTemperature: \n  - Current: 17.26°C\n  - High: 18.57°C\n  - Low: 16.2°C\n  - Feels like: 15.54°C\nRain: {}\nHeat index: None\nCloud cover: 0%', name='open_weather_map')]}

---

Output from node 'agent':
---
{'messages': [AIMessage(content='The current temperature in Las Vegas is 17.26°C. It feels like 15.54°C with clear skies and a humidity of 19%.', response_metadata={'finish_reason': 'stop'}, id='run-74ca072c-11b7-4a94-87d6-00fcf3a165e1-0')]}

---



In [57]:
from langchain_core.messages import HumanMessage
inputs = {"messages": [HumanMessage(content="How are you?")]}
for output in app.stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'agent':
---
{'messages': [AIMessage(content="I'm just a computer program, so I don't have feelings, but I'm here to help you with any questions or tasks you have. How can I assist you today?", response_metadata={'finish_reason': 'stop'}, id='run-13bf4c50-6360-4fd9-b34a-1312fe1129c3-0')]}

---

